In [2]:
from dataset_br.images_invalid import get_list_of_images_invalid
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import numpy as np
import itertools
import os
import pandas as pd
import sqlalchemy as sa
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

from tables import County, DataTrustedIdentifier, InfoImage, create_info_image
from database import connect, state_province_in_list_uf_or_list_state, has_state_in_locality
from sqlalchemy import and_, or_
from sqlalchemy.orm import Session

from unaccent import unaccent

def text_bold(string):
    return '\033[1m' + string + '\033[0m'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
session: Session
engine, session = connect()
engine.echo=False

2023-03-19 21:43:21,772 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-19 21:43:21,773 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 21:43:21,774 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-19 21:43:21,774 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 21:43:21,775 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-19 21:43:21,775 INFO sqlalchemy.engine.Engine [raw sql] {}


## regioes

In [4]:
total = 0
total_locality = 0
for regiao in ['Norte', 'Nordeste', 'Centro-Oeste', 'Sudeste', 'Sul']:
    county = session.query(County)\
        .filter(County.regiao==regiao)\
        .distinct()\
        .all()

    list_uf = [unaccent(sa.func.lower(c.uf)) for c in county]
    list_state = [unaccent(sa.func.lower(c.state)) for c in county]
    list_county = [unaccent(sa.func.lower(c.county)) for c in county]

    query_lower_unaccent_like = session.query(sa.func.concat('%', unaccent(sa.func.lower(County.state)), '%'))\
        .filter(County.regiao==regiao)\
        .distinct()\
        .all()

    list_state_like = [q[0] for q in query_lower_unaccent_like]
    columns = [DataTrustedIdentifier.specific_epithet, DataTrustedIdentifier.barcode, DataTrustedIdentifier.country, DataTrustedIdentifier.state_province, DataTrustedIdentifier.county]

    query_only_state = session.query(*columns)\
        .filter(and_(DataTrustedIdentifier.country_trusted =='Brasil',
                    DataTrustedIdentifier.specific_epithet.is_not(None),
                    state_province_in_list_uf_or_list_state(list_state, list_uf)))\
        .distinct()\
        .all()


    query_has_state_in_locality = session.query(*columns)\
        .filter(and_(DataTrustedIdentifier.country_trusted=='Brasil',
                     DataTrustedIdentifier.specific_epithet.is_not(None),
                     or_(state_province_in_list_uf_or_list_state(list_state, list_uf),
                         has_state_in_locality(list_state_like)
                         )))\
        .distinct()\
        .all()


    print('quantidade de registros usando somente %s, na regiao %s: %d' % (text_bold('state_province'), text_bold(regiao), len(query_only_state)))
    print('quantidade de registros usando somente %s, na regiao %s: %d' % (text_bold('state_province e locality'), text_bold(regiao), len(query_has_state_in_locality)))
    total = total + len(query_only_state)
    total_locality = total_locality + len(query_has_state_in_locality)

print('total sem %s: %d' % (text_bold('locality'), total))
print('total comm %s: %d' % (text_bold('locality'), total_locality))

/tmp/ipykernel_568402/1114647579.py:13: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  query_lower_unaccent_like = session.query(sa.func.concat('%', unaccent(sa.func.lower(County.state)), '%'))\


quantidade de registros usando somente state_province, na regiao Norte: 3186
quantidade de registros usando somente state_province e locality, na regiao Norte: 3676
quantidade de registros usando somente state_province, na regiao Nordeste: 1198
quantidade de registros usando somente state_province e locality, na regiao Nordeste: 1239
quantidade de registros usando somente state_province, na regiao Centro-Oeste: 1261
quantidade de registros usando somente state_province e locality, na regiao Centro-Oeste: 1282
quantidade de registros usando somente state_province, na regiao Sudeste: 3619
quantidade de registros usando somente state_province e locality, na regiao Sudeste: 3642
quantidade de registros usando somente state_province, na regiao Sul: 2651
quantidade de registros usando somente state_province e locality, na regiao Sul: 2737
total sem locality: 11915
total comm locality: 12576


In [5]:
def lsas(qz):
    l_species = []
    l_paths = []
    lc = []
    color = 'rgb'
    image_size = '512'
    sum = 0
    sumb = 0
    for i, qq in enumerate(qz):
        species = qq[0]
        list_seq = qq[1]
        qzinho = session.query(sa.func.array_agg(sa.distinct(InfoImage.path_image))) \
            .filter(and_(InfoImage.seq_id.in_(list_seq),
                         InfoImage.image_size == image_size,
                         InfoImage.color_mode == color)) \
            .group_by(InfoImage.seq_id) \
            .all()

        lsa = list(itertools.chain(*qzinho))  # "remove of tuples"

        if len(lsa) >= 5:
            # remove images duplicates
            lb = []
            for lx in lsa:
                lb.append(sorted(lx)[0])  # sorted and catch first value of list

            if len(np.unique(lb)) >= 5:
                l_species.append(species)
                l_paths.append(np.unique(lb).tolist())
                lc.append(len(np.unique(lb)))
                sumb += len(np.unique(lb))
                sum += 1
        # break

    # sum, sumb
    return l_species, l_paths


def remove_images_not_valid(list_species, list_path):
    list_path_correct = []
    list_count_images = []
    list_images_invalid = get_list_of_images_invalid()

    for i, s in enumerate(list_path):
        matching = [sx for sx in s if not any(xs in sx for xs in list_images_invalid['barcode'])]

        if len(s) != len(matching):
            print('specie: %s before: %d after: %d' % (list_species[i], len(s), len(matching)))
            diff = list(set(s) ^ set(matching))
            print('diff: %s' % str(diff))

        list_path_correct.append(matching)
        list_count_images.append(len(matching))

    return list_species, list_path_correct, list_count_images


columns = [DataTrustedIdentifier.specific_epithet, sa.func.array_agg(DataTrustedIdentifier.seq)]

for regiao in ['Norte', 'Nordeste', 'Centro-Oeste', 'Sudeste', 'Sul']:
    county = session.query(County) \
        .filter(County.regiao == regiao) \
        .distinct() \
        .all()

    list_uf = [unaccent(sa.func.lower(c.uf)) for c in county]
    list_state = [unaccent(sa.func.lower(c.state)) for c in county]
    list_county = [unaccent(sa.func.lower(c.county)) for c in county]

    # columns = [DataTrustedIdentifier.specific_epithet, DataTrustedIdentifier.barcode, DataTrustedIdentifier.country, DataTrustedIdentifier.state_province, DataTrustedIdentifier.county]

    query_only_state = session.query(*columns) \
        .filter(and_(DataTrustedIdentifier.country_trusted == 'Brasil',
                     DataTrustedIdentifier.specific_epithet.is_not(None),
                     state_province_in_list_uf_or_list_state(list_state, list_uf))) \
        .distinct() \
        .group_by(DataTrustedIdentifier.specific_epithet) \
        .order_by(DataTrustedIdentifier.specific_epithet) \
        .having(sa.func.count(DataTrustedIdentifier.specific_epithet) >= 5) \
        .all()

    list_species, list_path = lsas(query_only_state)
    list_species, list_path, list_count = remove_images_not_valid(list_species, list_path)

    display(pd.DataFrame({
        'species': list_species,
        'paths': list_path,
        'count': list_count
    }))
    break

['/home/xandao/Documentos/RGB/512/w_pred_mask/CEN00094895.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/CEN00094784.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/IAN033182.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY01421130_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY01421132_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY01421157_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY01421137_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY01421158_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY01421092_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY00756336_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY00957084_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY01421159_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY00756252_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY01075709_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY01075081_01.jpeg', '/home/xandao/Doc

,species,paths,count
0,aduncum,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...,48
1,aequale,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...,9
2,alata,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...,8
3,alatabaccum,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...,28
4,aleyreanum,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...,48
...,...,...,...
102,urocarpa,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...,6
103,vermiculatum,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...,9
104,victoriana,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...,5
105,vitaceum,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...,13
